In [39]:
from pathlib import Path
import pandas as pd
import numpy as np
from scipy.signal import hilbert
from mne_bids import BIDSPath, read_raw_bids
import mne
from mne.preprocessing import compute_proj_ecg, compute_proj_eog
from mne.minimum_norm import apply_inverse_epochs, make_inverse_operator

## Load raw data

In [2]:
# paths
DATA_DIR = Path("/storage/store/data")
BIDS_ROOT = DATA_DIR / "camcan/BIDSsep/smt/"
SSS_CAL_FILE = DATA_DIR / "camcan-mne/Cam-CAN_sss_cal.dat"
CT_SPARSE_FILE = DATA_DIR / "camcan-mne/Cam-CAN_ct_sparse.fif"
PARTICIPANTS_FILE = BIDS_ROOT / "participants.tsv"

In [3]:
# all participants
participants = pd.read_csv(PARTICIPANTS_FILE, sep='\t', header=0)
participants

,participant_id,age,sex,hand,Unnamed: 4
0,sub-CC110033,24.17,MALE,90.0,NaN
1,sub-CC110037,18.75,MALE,89.0,NaN
2,sub-CC110045,24.50,FEMALE,100.0,NaN
3,sub-CC110056,22.67,FEMALE,100.0,NaN
4,sub-CC110069,28.67,FEMALE,-58.0,NaN
...,...,...,...,...,...
638,sub-CC722542,79.33,MALE,100.0,NaN
639,sub-CC722651,79.83,FEMALE,100.0,NaN
640,sub-CC722891,84.42,FEMALE,89.0,NaN
641,sub-CC723197,80.08,FEMALE,100.0,NaN


In [4]:
# read raw data of one participant
subject = "CC110033"

bp = BIDSPath(
    root=BIDS_ROOT,
    subject=subject,
    task="smt",
    datatype="meg",
    extension=".fif",
    session="smt",
)   
raw = read_raw_bids(bp)

Opening raw data file /storage/store/data/camcan/BIDSsep/smt/sub-CC110033/ses-smt/meg/sub-CC110033_ses-smt_task-smt_meg.fif...
    Read a total of 8 projection items:
        mag_ssp_upright.fif : PCA-mags-v1 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v2 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v3 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v4 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v5 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-grad-v1 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-grad-v2 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-grad-v3 (1 x 306)  idle
    Range : 16000 ... 557999 =     16.000 ...   557.999 secs
Ready.
Reading events from /storage/store/data/camcan/BIDSsep/smt/sub-CC110033/ses-smt/meg/sub-CC110033_ses-smt_task-smt_events.tsv.
Reading channel info from /storage/store/data/camcan/BIDSsep/smt/sub-CC110033/ses-smt/meg/sub-CC110033_ses-smt_task-smt_channels.tsv.
Not fully anonymizing info - keeping h

/tmp/ipykernel_3050/848390475.py:12: RuntimeWarning: Unable to map the following column(s) to to MNE:
hand (subject handedness): 90
: 
  raw = read_raw_bids(bp)


In [5]:
print(f"Duration: {raw.times[-1]} s.")

Duration: 541.999 s.


## Preprocessing used in the code sent by Alex

In [6]:
# preprocess
go = False
if go:
    raw.crop(0, 500).pick_types(meg=True, eeg=False).load_data().resample(80)
    raw.apply_gradient_compensation(3)
    projs_ecg, _ = compute_proj_ecg(raw, n_grad=1, n_mag=2)
    projs_eog, _ = compute_proj_eog(raw, n_grad=1, n_mag=2, ch_name="MLT31-4407")
    raw.add_proj(projs_ecg + projs_eog)
    raw.apply_proj()
    raw.filter(0.1, None)  # this helps with symmetric orthogonalization later
    cov = mne.compute_raw_covariance(raw)  # compute before band-pass of interest

- apply_gradient_compensation(3) doesn't work
- the channel name "ch_name='MLT31-4407'" is not known and no EOG is found if we remove it

## Preprocessing used by Cédric

In [7]:
# preprocess
raw.load_data()
raw.filter(l_freq=None, h_freq=125)
raw.notch_filter([50, 100])
raw = mne.preprocessing.maxwell_filter(
    raw, calibration=SSS_CAL_FILE, cross_talk=CT_SPARSE_FILE, st_duration=10.0)

Reading 0 ... 541999  =      0.000 ...   541.999 secs...
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 1.2e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 125.00 Hz
- Upper transition bandwidth: 31.25 Hz (-6 dB cutoff frequency: 140.62 Hz)
- Filter length: 107 samples (0.107 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.6s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    2.5s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    5.7s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:   10.1s


Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 6601 samples (6.601 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    2.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    4.5s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    8.0s


Maxwell filtering raw data
102 of 102 magnetometer types replaced with T3.
    No bad MEG channels
    Processing 204 gradiometers and 102 magnetometers
    Using fine calibration Cam-CAN_sss_cal.dat
        Adjusting non-orthogonal EX and EY
        Adjusted coil orientations by (μ ± σ): 0.3° ± 0.3° (max: 1.3°)
    Automatic origin fit: head of radius 92.9 mm
    Using origin 7.2, 1.3, 26.2 mm in the head frame
    Processing data using tSSS with st_duration=10.0
        Using 86/95 harmonic components for    0.000  (71/80 in, 15/15 out)
    Using loaded raw data
    Spatiotemporal window did not fit evenly intocontiguous data segment. 2.00 seconds were lumped into the previous window.
    Processing 54 data chunks
        Projecting  0 intersecting tSSS components for    0.000 -    9.999 s  (#1/54)
        Projecting  0 intersecting tSSS components for   10.000 -   19.999 s  (#2/54)
        Projecting  2 intersecting tSSS components for   20.000 -   29.999 s  (#3/54)
        Projecti

In [8]:
# events
all_events, all_event_id = mne.events_from_annotations(raw)
metadata_tmin, metadata_tmax = -3., 0
metadata, events, event_id = mne.epochs.make_metadata(
    events=all_events, event_id=all_event_id,
    tmin=metadata_tmin, tmax=metadata_tmax, sfreq=raw.info['sfreq'],
    row_events="button", 
    keep_last="audiovis",
)

Used Annotations descriptions: ['audiovis/1200Hz', 'audiovis/300Hz', 'audiovis/600Hz', 'button', 'catch/0', 'catch/1']


In [9]:
# epochs
tmin = -1.7
tmax = 1.7
baseline = (-1.25, -1.0)
epochs = mne.Epochs(
    raw, events, event_id, metadata=metadata,
    tmin=tmin, tmax=tmax,
    baseline=baseline,
    preload=True, verbose=False
)

In [10]:
# button event is at most one sec. after an audiovis event,
# and with at least 3 sec. between 2 button events.
epochs = epochs["event_name == 'button' and audiovis > -1. and button == 0."]

In [34]:
epochs

<Epochs | 62 events (all good), -1.7 – 1.7 s (baseline -1.25 – -1 s), ~551.7 MiB, data loaded, with metadata,
 'button': 62>

## Inverse model

In [20]:
# subject_with_src = [
#     "CC120727", "CC222185", "CC320776", "CC321880", "CC410084", "CC520002",
#     "CC520287", "CC520552", "CC620085", "CC721107"]
SRC = DATA_DIR / "camcan-mne/freesurfer/fsaverage/bem/fsaverage-oct-5-src.fif"
TRANS_DIR = DATA_DIR / "camcan-mne/trans"
trans = TRANS_DIR / f"sub-{subject}-trans.fif"
BEM_DIR = DATA_DIR / "camcan-mne/freesurfer"
bem = BEM_DIR / f"{subject}/bem/{subject}-meg-bem.fif"
# bem = BEM_DIR / "fsaverage/bem/fsaverage-5120-5120-5120-bem-sol.fif"  # makes forward model fail

In [21]:
src = mne.read_source_spaces(SRC)
fwd = mne.make_forward_solution(raw.info, trans, src, bem, verbose=True)
del src
cov = mne.compute_raw_covariance(raw)
inv = make_inverse_operator(raw.info, fwd, cov)
del fwd

    Reading a source space...
    [done]
    Reading a source space...
    [done]
    2 source spaces read
Source space          : <SourceSpaces: [<surface (lh), n_vertices=163842, n_used=1026>, <surface (rh), n_vertices=163842, n_used=1026>] MRI (surface RAS) coords, subject 'fsaverage', ~21.3 MiB>
MRI -> head transform : /storage/store/data/camcan-mne/trans/sub-CC110033-trans.fif
Measurement data      : instance of Info
Conductor model   : /storage/store/data/camcan-mne/freesurfer/CC110033/bem/CC110033-meg-bem.fif
Accurate field computations
Do computations in head coordinates
Free source orientations

Read 2 source spaces a total of 2052 active source locations

Coordinate transformation: MRI (surface RAS) -> head
    0.997089 -0.050235 -0.057351       5.61 mm
    0.061161 0.976153 0.208292      26.11 mm
    0.045520 -0.211193 0.976384      49.85 mm
    0.000000 0.000000 0.000000       1.00

Read 306 MEG channels from info
105 coil definitions read
Coordinate transformation: MEG dev

## Source Time Courses (STCs)

In [22]:
# evoked = epochs.average()
# evoked.plot()

In [23]:
FREESURFER_DIR = DATA_DIR / "camcan-mne/freesurfer"
labels = mne.read_labels_from_annot(subject, "aparc", subjects_dir=FREESURFER_DIR)

Reading labels from parcellation...
   read 34 labels from /storage/store/data/camcan-mne/freesurfer/CC110033/label/lh.aparc.annot
   read 34 labels from /storage/store/data/camcan-mne/freesurfer/CC110033/label/rh.aparc.annot


In [26]:
# too long! I stopped the execution after 5 minutes
# src = mne.setup_source_space(subject, subjects_dir=FREESURFER_DIR)

In [27]:
stcs = apply_inverse_epochs(
    epochs, inv, lambda2=1.0 / 9.0, pick_ori="normal", return_generator=True
)

In [28]:
# morph subject-specific labels to fsaverage
morphed_labels = mne.morph_labels(
    labels, subject_to="fsaverage", subject_from="CC110033", subjects_dir=FREESURFER_DIR
)
label_ts = mne.extract_label_time_course(
    stcs, morphed_labels, inv["src"], return_generator=False
)

    Left-hemisphere map read.
    Right-hemisphere map read.
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    The projection vectors do not apply to these channels.
    Created the whitener using a noise covariance matrix with rank 71 (235 small eigenvalues omitted)
    Computing noise-normalization factors (dSPM)...
[done]
Picked 306 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 62
Extracting time courses for 68 labels (mode: mean_flip)
Processing epoch : 2 / 62
Extracting time courses for 68 labels (mode: mean_flip)
Processing epoch : 3 / 62
Extracting time courses for 68 labels (mode: mean_flip)
Processing epoch : 4 / 62
Extracting time courses for 68 labels (mode: mean_flip)
Processing epoch : 5 / 62
Extracting time courses for 68 labels (mode: mean_flip)
Processing epoch : 6 / 62
Extracting time courses for 68 labels (mode: 

In [33]:
np.array(label_ts).shape  # shape (n_epochs, n_labels, n_timecourses)

(62, 68, 3401)

In [ ]:
# # OR morph source estimate to subject space
# morphed_stcs = [
#     stc.morph(subject_to="CC110033", subject_from="fsaverage", subjects_dir=FREESURFER_DIR)
#     for stc in stcs
# ]
# label_ts = mne.extract_label_time_course(
#     morphed_stcs, labels, inv["src"], return_generator=False
# )

In [32]:
# error because of subject-specific labels and fsaverage stcs
# label_ts = mne.extract_label_time_course(
#     stcs, labels, inv["src"], return_generator=False
# )
# del stcs

## Envelopes

In [35]:
sfreq = epochs.info["sfreq"]

In [36]:
# # we can eventually band-pass time courses in beta waves range
# def bp_gen(label_ts):
#     """Make a generator that band-passes on the fly."""
#     for ts in label_ts:
#         yield mne.filter.filter_data(ts, sfreq, 14, 30)

In [43]:
# Apply Hilbert transform to compute the analytic signal
hilbert_ts = hilbert(label_ts, axis=2)
envelope = np.abs(hilbert_ts)

In [45]:
envelope.shape

(62, 68, 3401)

## plot

In [48]:
import mne_connectivity
from mne_connectivity import envelope_correlation

In [49]:
# we can eventually band-pass time courses in beta waves range
def bp_gen(label_ts):
    """Make a generator that band-passes on the fly."""
    for ts in label_ts:
        yield mne.filter.filter_data(ts, sfreq, 14, 30)

In [50]:
# compute symmetric-orthogonalized envelope correlation
label_ts_orth = mne_connectivity.envelope.symmetric_orth(label_ts)
corr_obj = envelope_correlation(  # already orthogonalized earlier
    bp_gen(label_ts_orth), orthogonalize=False
)

# average over epochs, take absolute value, and plot
corr = corr_obj.combine()
corr = corr.get_data(output="dense")[:, :, 0]
corr.flat[:: corr.shape[0] + 1] = 0  # zero out the diagonal
corr = np.abs(corr)

Setting up band-pass filter from 14 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 14.00
- Lower transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 12.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 943 samples (0.943 s)

Setting up band-pass filter from 14 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 14.00
- Lower transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 12.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Setting up band-pass filter from 14 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 14.00
- Lower transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 12.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


- Filter length: 943 samples (0.943 s)

Setting up band-pass filter from 14 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 14.00
- Lower transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 12.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 943 samples (0.943 s)



...

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Setting up band-pass filter from 14 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 14.00
- Lower transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 12.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 943 samples (0.943 s)

Setting up band-pass filter from 14 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 14.00
- Lower transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 12.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Setting up band-pass filter from 14 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 14.00
- Lower transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 12.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 943 samples (0.943 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
